In [3]:
# --- 1. Import Libraries ---
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [4]:
# --- 2. Load and Prepare Data ---
df = pd.read_csv('global_energy_consumption.csv')
df_model = df.drop(['Year'], axis=1)

In [5]:
# --- 3. Simulate Missing Features ---
df_model['Simulated_Population'] = df_model['Total Energy Consumption (TWh)'] / df_model['Per Capita Energy Use (kWh)']
df_model['Simulated_GDP'] = df_model['Simulated_Population'] * df_model['Energy Price Index (USD/kWh)']


In [6]:
# --- 4. Feature Engineering ---
df_model['Fossil_Energy_Used'] = df_model['Total Energy Consumption (TWh)'] * (df_model['Fossil Fuel Dependency (%)'] / 100)
df_model['Industrial × Fossil'] = df_model['Industrial Energy Use (%)'] * df_model['Fossil Fuel Dependency (%)']
df_model['Household × Price'] = df_model['Household Energy Use (%)'] * df_model['Energy Price Index (USD/kWh)']
df_model['Energy_Intensity'] = df_model['Total Energy Consumption (TWh)'] / df_model['Per Capita Energy Use (kWh)']
df_model['Fossil_Intensity'] = df_model['Fossil_Energy_Used'] / df_model['Per Capita Energy Use (kWh)']

In [7]:
# --- 5. Define New Target: Carbon Intensity ---
df_model['Carbon_Intensity'] = df_model['Carbon Emissions (Million Tons)'] / df_model['Total Energy Consumption (TWh)']
y = np.log1p(df_model['Carbon_Intensity'])


In [8]:
# --- 6. Define Features ---
X = df_model.drop(columns=['Country', 'Carbon Emissions (Million Tons)', 'Carbon_Intensity'])

In [9]:
# --- 7. Scale Features ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# --- 8. Train-Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# --- 9. Train Gradient Boosting Model ---
gb = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=6, n_estimators=300,
                          random_state=42)

In [12]:
# --- 10. Predict and Reverse Log Transform ---
y_pred_log = gb.predict(X_test)
y_test_exp = np.expm1(y_test)
y_pred_exp = np.expm1(y_pred_log)

In [13]:
# --- 11. Evaluate Model ---
def evaluate(y_true, y_pred, model_name):
    print(f"\n{model_name} Results:")
    print(f"R² Score: {r2_score(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print("-" * 30)

In [14]:
evaluate(y_test_exp, y_pred_exp, "Gradient Boosting (Carbon Intensity)")


Gradient Boosting (Carbon Intensity) Results:
R² Score: 0.6484
RMSE: 1.3719
------------------------------


In [15]:
# --- Predict Function ---
def predict_carbon_intensity(input_dict):
    """
    Predict carbon intensity from new input data.
    input_dict: dictionary with keys matching feature names
    Returns: predicted carbon intensity (Million Tons per TWh)
    """
    # Convert input to DataFrame
    input_df = pd.DataFrame([input_dict])

    # --- Simulate missing features ---
    input_df['Fossil_Energy_Used'] = input_df['Total Energy Consumption (TWh)'] * (input_df['Fossil Fuel Dependency (%)'] / 100)
    input_df['Industrial × Fossil'] = input_df['Industrial Energy Use (%)'] * input_df['Fossil Fuel Dependency (%)']
    input_df['Household × Price'] = input_df['Household Energy Use (%)'] * input_df['Energy Price Index (USD/kWh)']
    input_df['Energy_Intensity'] = input_df['Total Energy Consumption (TWh)'] / input_df['Per Capita Energy Use (kWh)']
    input_df['Fossil_Intensity'] = input_df['Fossil_Energy_Used'] / input_df['Per Capita Energy Use (kWh)']
    input_df['Simulated_Population'] = input_df['Total Energy Consumption (TWh)'] / input_df['Per Capita Energy Use (kWh)']
    input_df['Simulated_GDP'] = input_df['Simulated_Population'] * input_df['Energy Price Index (USD/kWh)']

    # Drop unused columns
    input_df = input_df[X.columns]  # match training features

    # Scale input
    input_scaled = scaler.transform(input_df)

    # Predict log carbon intensity
    pred_log = gb.predict(input_scaled)

    # Reverse log transform
    pred_intensity = np.expm1(pred_log)

    return round(pred_intensity[0], 4)

In [17]:
sample_input = {
    'Total Energy Consumption (TWh)': 1200,
    'Fossil Fuel Dependency (%)': 70,
    'Industrial Energy Use (%)': 40,
    'Household Energy Use (%)': 30,
    'Per Capita Energy Use (kWh)': 5000,
    'Energy Price Index (USD/kWh)': 0.12,
    'Renewable Energy Share (%)': 25  
}

predicted_intensity = predict_carbon_intensity(sample_input)
print("Predicted Carbon Intensity:", predicted_intensity, "Million Tons per TWh")

Predicted Carbon Intensity: 1.579 Million Tons per TWh
